In [0]:
%pip install scipy==1.13.0
%pip install pandas==2.1.4
%pip install numpy==1.26.0
%pip install skforecast==0.12.1
%pip install scikit-learn==1.4.0
%pip install "mlflow-skinny[databricks]>=2.4.1"
%pip install scikeras==0.1.8
%pip install openpyxl==3.1.2
%pip install feature-engine==1.7.0
%pip install autots==0.6.14

%pip install --upgrade threadpoolctl
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 57.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Not uninstalling numpy at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.1
    Not uninstalling scipy at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9
    Can't uninstall 'scipy'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated pac

In [0]:
# Import processig libraries
import pandas as pd
import numpy as np
import pyspark.pandas as pf
from autots import AutoTS

#graphics
import matplotlib.pyplot as plt
import plotly.express as px

#metrics sklearn
import scipy.stats as st
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, r2_score 
import ast
from sklearn.impute import KNNImputer


#config
import datetime
import joblib
import logging

# Set up logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)

# Update custom parameters
plt.rcParams.update({
    'font.family': 'serif',  
    'font.size': 8,          
    'axes.labelsize': 14,    
    'axes.titlesize': 16,    
    'lines.linewidth': 1.5,  
    'figure.dpi': 100})

In [0]:
# Define the paths for the bronze, silver, and gold storage locations
path_bronce = 'abfss://bronze@grupoisastorageimg02.dfs.core.windows.net/Mercados'
path_silver = 'abfss://lz-datos-limpios@grupoisastorageimg02.dfs.core.windows.net/Mercados'
path_gold = 'abfss://lz-datos-productivo@grupoisastorageimg02.dfs.core.windows.net/Mercados'

In [0]:
#reading imports
df = spark.read.options(header='true',sep=',').csv(f'{path_bronce}/exogenas_zinc.csv').toPandas()
df.rename(columns={"Period":"Date"}, inplace=True)
indices_to_drop = df.tail(4).index
#drop last 4 files
df = df.drop(indices_to_drop)

## Cleaning NE Import and Write it in lz.limpios.

In [0]:
data_exog = df.copy(deep=True)
data_exog.head()

,Date,coil_import_SA_tonne,coil_import_China_tonne,coil_import_NE_tonne,coil_export_India_tonne
0,Apr 2024,730,4698.75,768.13,791.88
1,Mar 2024,739.5,4777,793.75,816
2,Feb 2024,760,4961.67,820,847.5
3,Jan 2024,756.25,5068.75,816,832.5
4,Dec 2023,751.5,5116,794.38,824


In [0]:
# Add pd.offsets.MonthEnd(0) to the "Date" column in the data_exog dataframe
data_exog['Date']= pd.to_datetime(data_exog["Date"]) + pd.offsets.MonthEnd(0)
data_exog['Date']= data_exog.set_index('Date', inplace=True)
data_exog= data_exog[data_exog.index>='2008-01-31']
data_exog.head()

/home/spark-3a0927d3-b992-44c6-af64-25/.ipykernel/49059/command-797265371109162-2735164761:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_exog['Date']= pd.to_datetime(data_exog["Date"]) + pd.offsets.MonthEnd(0)


,coil_import_SA_tonne,coil_import_China_tonne,coil_import_NE_tonne,coil_export_India_tonne,Date
Date,,,,,
2024-04-30,730,4698.75,768.13,791.88,None
2024-03-31,739.5,4777,793.75,816,None
2024-02-29,760,4961.67,820,847.5,None
2024-01-31,756.25,5068.75,816,832.5,None
2023-12-31,751.5,5116,794.38,824,None


In [0]:
#check for missing values, and save it in catalog
exog_filtered = data_exog[['coil_import_NE_tonne']]
exog_filtered = exog_filtered.rename(columns={'coil_import_NE_tonne':'import'})

In [0]:
exog_filtered.isnull().sum()

import    0
dtype: int64

In [0]:
exog_filtered = exog_filtered.reset_index()
exog_filtered.head()

,Date,import
0,2024-04-30,768.13
1,2024-03-31,793.75
2,2024-02-29,820
3,2024-01-31,816
4,2023-12-31,794.38


In [0]:
# # Export result to new table
# dataframe = spark.createDataFrame(exog_filtered)
# dataframe.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lz_limpio.mercados.coilimportNE")

## Reading and pre-processing

In [0]:
ipc_china = spark.sql("""
        SELECT Date, ipc_china
        FROM lz_productivo.mercados.pronosticos_ipcchinamensual_from2008
    """).toPandas()# Convert the 'Date' column to datetime format
ipc_china["Date"] = pd.to_datetime(ipc_china["Date"])
ipc_china = ipc_china.set_index('Date').sort_index()
ipc_china_to_train = pd.DataFrame()
ipc_china_to_train = ipc_china[ipc_china.index<='2024-04-30']


In [0]:
Selected_features = ['coil_import_NE_tonne', 'Ipc_India', 'Aluminio']

In [0]:
ipc_india = spark.sql("""
        SELECT Date, Mid
        FROM lz_productivo.mercados.pronosticos_ipcindiamensual_from2008
    """).toPandas()# Convert the 'Date' column to datetime format
ipc_india["Date"] = pd.to_datetime(ipc_india["Date"])
ipc_india = ipc_india.rename(columns={'Mid':'ipc_india'})
ipc_india = ipc_india.set_index('Date').sort_index()
ipc_india_to_train = pd.DataFrame()
ipc_india_to_train_india = ipc_india[ipc_india.index<='2024-04-30']

In [0]:
import_NE = spark.sql("""
        SELECT Date, import
        FROM lz_limpio.mercados.coilimportne
    """).toPandas()# Convert the 'Date' column to datetime format
import_NE["Date"] = pd.to_datetime(import_NE["Date"])
import_NE = import_NE.set_index('Date').sort_index()

In [0]:
aluminio = spark.sql("""
        SELECT Date, Mid
        FROM lz_productivo.mercados.pronosticos_aluminiomensualvalorpromedio
    """).toPandas()# Convert the 'Date' column to datetime format
aluminio["Date"] = pd.to_datetime(aluminio["Date"]) + pd.offsets.MonthEnd(0)
aluminio = aluminio.rename(columns={'Mid':'aluminio'})
aluminio = aluminio.set_index('Date').sort_index()
start_date = '2008-01-31'
end_date = '2024-04-30'
aluminio_to_train = pd.DataFrame()
aluminio_to_train = aluminio.loc[start_date:end_date]
aluminio_to_predict = pd.DataFrame()
aluminio_to_predict = aluminio[aluminio.index>'2024-04-30']

In [0]:
aluminio_to_train.tail()

,aluminio
Date,
2023-12-31,1792.768720
2024-01-31,1803.994535
2024-02-29,1764.944210
2024-03-31,1739.765402
2024-04-30,1688.010683


In [0]:
# Read the table into a Spark DataFrame and convert it to a Pandas DataFrame
ts = spark.read.table(f'lz_limpio.mercados.precio_zinc_mensualvalorpromedio').toPandas()
# Convert the 'Date' column to datetime format
ts["Date"] = pd.to_datetime(ts["Date"])
ts = ts.set_index('Date').sort_index()
ts = ts[ts.index>='2008-01-31']
# Display the updated DataFrame
ts.head(5)

,mean
Date,
2008-01-31,2339.806818
2008-02-29,2439.214286
2008-03-31,2511.026316
2008-04-30,2263.170455
2008-05-31,2181.625000


In [0]:
#MinMaxScaler
scaled_data = ts.copy()
#convert data into matrix of row-col vectors
print('zinc.values:', len(scaled_data.values))
scaled_data =  scaled_data.values.reshape((len(scaled_data), 1))
#feature scaling
scaler = MinMaxScaler(feature_range=(0, 1))
#fit the scaler with the train data to get min-max values
scaler = scaler.fit(scaled_data)
print("min and max: " ,scaler.data_min_, scaler.data_max_)
#normalize the data and sanity check
normalized_commodity = scaler.transform(scaled_data)
normalized_list = [normalized_commodity[i] for i in range(5)]

zinc.values: 196
min and max:  [1100.01190476] [4370.40789474]


In [0]:
def create_time_features(df, target_colum,lags=1):

    # Create time-based features
    df['month'] = df.index.month
    df['day_of_month'] = df.index.day
    
    #adding IPC_lag 23
    df['ipc_china'] = ipc_china_to_train['ipc_china'] 
    df['ipc_india'] = ipc_india_to_train_india['ipc_india']
    df['import'] = import_NE['import']
    df['aluminio'] = aluminio_to_train['aluminio']

    # Create lagged features
    for lag in range(1, lags + 1):
        df[f'lag_{lag}'] = df[target_colum].shift(lag)  # Shift target column by lag
        
    # Check for null values in the lagged columns and delete rows with null values
    rows_before = df.shape[0]  # Number of rows before dropping null values
    df.dropna(subset=[f'lag_{lag}' for lag in range(1, lags + 1)], inplace=True)
    rows_after = df.shape[0]  # Number of rows after dropping null values

    print(f"{rows_before - rows_after} rows were deleted due to null values in lagged columns.")
    return df

In [0]:
normalized_series = pd.DataFrame(normalized_commodity, index=ts.index).sort_index()
target_column = 0
data_ready = create_time_features(normalized_series,target_column, lags=10)  
data_ready.head()

10 rows were deleted due to null values in lagged columns.


,0,month,day_of_month,ipc_china,ipc_india,import,aluminio,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10
Date,,,,,,,,,,,,,,,,,
2008-11-30,0.015946,11,30,91.297822,68.135826,865,1852.075000,0.061075,0.194173,0.188712,0.227899,0.242751,0.330728,0.355663,0.431451,0.409492,0.379096
2008-12-31,0.000000,12,31,91.669473,67.177124,682,1490.023810,0.015946,0.061075,0.194173,0.188712,0.227899,0.242751,0.330728,0.355663,0.431451,0.409492
2009-01-31,0.026631,1,31,92.387070,66.514909,545,1412.785714,0.000000,0.015946,0.061075,0.194173,0.188712,0.227899,0.242751,0.330728,0.355663,0.431451
2009-02-28,0.003589,2,28,93.028409,66.935699,530,1329.812500,0.026631,0.000000,0.015946,0.061075,0.194173,0.188712,0.227899,0.242751,0.330728,0.355663
2009-03-31,0.035963,3,31,92.419349,67.335350,462.5,1335.500000,0.003589,0.026631,0.000000,0.015946,0.061075,0.194173,0.188712,0.227899,0.242751,0.330728


In [0]:
data_ready.index.min(), data_ready.index.max()

(Timestamp('2008-11-30 00:00:00'), Timestamp('2024-04-30 00:00:00'))

In [0]:
data_ready.isnull().sum()

0               0
month           0
day_of_month    0
ipc_china       0
ipc_india       0
import          0
aluminio        0
lag_1           0
lag_2           0
lag_3           0
lag_4           0
lag_5           0
lag_6           0
lag_7           0
lag_8           0
lag_9           0
lag_10          0
dtype: int64

## Iterating with AutoTs, Find Best Model

In [0]:
horizon_prediction = 0
train_size = int(len(data_ready)-horizon_prediction)
# splitting the dataset in trian and test
train_series = data_ready.iloc[:train_size]
test_series =  data_ready.iloc[train_size:]
len(test_series), len(train_series)

(0, 186)

In [0]:
# Define target variable (assuming it's a column in the DataFrame)
train_target = train_series[target_column]
test_target = test_series[target_column]
# Extracting features (excluding the target column)
train_features = train_series.drop(target_column, axis=1)
test_features = test_series.drop(target_column, axis=1)
test_target.head()

Series([], Name: 0, dtype: float64)

In [0]:
len(train_features), len(train_target)

(186, 186)

In [0]:
import time

# Assuming df_zinc is already defined and date is set as the index
model = AutoTS(
    forecast_length=60,
    frequency='infer',
    prediction_interval=0.9,
    ensemble=None,
    model_list="fast_parallel",  # Options: "superfast", "default", "fast_parallel"
    transformer_list="fast",  # Options: "superfast"
    max_generations=8,
    num_validations=2,
    constraint=2.0, 
    validation_method="backwards"
)

# Prepare the features (future_regressor)
features = train_features[:60]

# Measure the time taken to fit the model
start_time = time.time()
model = model.fit(
    train_target,
    #value_col=target_column,  # <-- This is where the target variable is specified
    future_regressor=features
)
end_time = time.time()
fit_time = end_time - start_time
print(f"Time taken to fit the model: {fit_time:.2f} seconds")

# Measure the time taken to predict
start_time = time.time()
prediction = model.predict(forecast_length=60, future_regressor=features)
end_time = time.time()
predict_time = end_time - start_time
print(f"Time taken to predict: {predict_time:.2f} seconds")

# Get the forecast
forecast = prediction.forecast
print(forecast)

# Calculate validation metrics (accuracy and prediction interval probability)
validation_results = model.results()
validation_accuracy = validation_results['smape'].mean()  # Mean SMAPE of the validations
prediction_interval_prob = model.prediction_interval  # Probability of prediction interval

print(f"Validation accuracy (SMAPE): {validation_accuracy:.2f}")
print(f"Prediction interval probability: {prediction_interval_prob}")


Using 2 cpus for n_jobs.
Data frequency is: ME, used frequency is: ME
future_regressor row count does not match length of training data


/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/autots/tools/shaping.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(self.na_strings, np.nan, inplace=True)  # pd.NA in future
/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/autots/tools/shaping.py:256: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore', downcast=self.downcast)


Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model ARIMA in generation 0 of 8
None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]
Model Number: 2 with model AverageValueNaive in generation 0 of 8
None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]
Model Number: 3 with model AverageValueNaive in generation 0 of 8
None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds'

/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]


...

None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]
184 - RRVAR with avg smape 58.7: 
Model Number: 185 of 187 with model RRVAR for Validation 1
None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]
185 - RRVAR with avg smape 59.06: 
Model Number: 186 of 187 with model RRVAR for Validation 1
None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_directio

/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]
187 - RRVAR with avg smape 199.99: 
Time taken to fit the model: 511.55 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/autots/evaluator/auto_model.py:2460: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_results.model_results = validation_results.model_results.replace(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0927d3-b992-44c6-af64-25d95c096fb9/lib/python3.10/site-packages/autots/evaluator/auto_model.py:2460: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_results.model_results = validation_results.model_results.replace

None
[{'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'upper', 'constraint_regularization': 1, 'bounds': False}, {'constraint_method': 'stdev_min', 'constraint_value': 2.0, 'constraint_direction': 'lower', 'constraint_regularization': 1, 'bounds': False}]
Time taken to predict: 0.15 seconds
                   0
2024-05-31  0.535977
2024-06-30  0.536795
2024-07-31  0.537532
2024-08-31  0.538216
2024-09-30  0.538881
2024-10-31  0.539434
2024-11-30  0.539955
2024-12-31  0.540344
2025-01-31  0.540637
2025-02-28  0.540982
2025-03-31  0.541064
2025-04-30  0.541082
2025-05-31  0.540919
2025-06-30  0.540681
2025-07-31  0.540244
2025-08-31  0.539660
2025-09-30  0.538986
2025-10-31  0.538090
2025-11-30  0.537098
2025-12-31  0.535871
2026-01-31  0.534472
2026-02-28  0.533111
2026-03-31  0.531361
2026-04-30  0.529504
2026-05-31  0.527391
2026-06-30  0.525171
2026-07-31  0.522690
2026-08-31  0.520025
2026-09-30  0.517258
2026-10-31  0.514228
2026-11-30  0.511103


In [0]:
# Retrieve and print the selected model details
best_model_info = model.best_model
print("Selected Model Information:")
print(best_model_info)

Selected Model Information:
                                   ID Model  \
422  6207cafff08ee9903558ecff1052eb59   FFT   

                             ModelParameters  \
422  {"n_harmonics": 2, "detrend": "linear"}   

                                                                                                                                                              TransformationParameters  \
422  {"fillna": "ffill", "transformations": {"0": "ClipOutliers", "1": "SinTrend"}, "transformation_params": {"0": {"method": "clip", "std_threshold": 3.5, "fillna": null}, "1": {}}}   

     Ensemble  
422         0  


In [0]:
best_model_info

,ID,Model,ModelParameters,TransformationParameters,Ensemble
422,6207cafff08ee9903558ecff1052eb59,FFT,"{""n_harmonics"": 2, ""detrend"": ""linear""}","{""fillna"": ""ffill"", ""transformations"": {""0"": ""ClipOutliers"", ""1"": ""SinTrend""}, ""transformation_params"": {""0"": {""method"": ""clip"", ""std_threshold"": 3.5, ""fillna"": null}, ""1"": {}}}",0


In [0]:
print("Selected Model Information 2:")
best_model_info

Selected Model Information 2:


,ID,Model,ModelParameters,TransformationParameters,Ensemble
1331,c7e1defbbc70802a088bada8c1478965,Cassandra,"{""preprocessing_transformation"": {""fillna"": ""f...","{""fillna"": ""ffill"", ""transformations"": {""0"": ""...",0


In [0]:

df =best_model_info.copy()

# Adjust pandas display options to show the full content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Print the entire DataFrame
print("Full DataFrame:")
df.head()


Full DataFrame:


,ID,Model,ModelParameters,TransformationParameters,Ensemble
422,6207cafff08ee9903558ecff1052eb59,FFT,"{""n_harmonics"": 2, ""detrend"": ""linear""}","{""fillna"": ""ffill"", ""transformations"": {""0"": ""ClipOutliers"", ""1"": ""SinTrend""}, ""transformation_params"": {""0"": {""method"": ""clip"", ""std_threshold"": 3.5, ""fillna"": null}, ""1"": {}}}",0
